In [ ]:
import pvdeg
from pvdeg import TEST_DATA_DIR
import pandas as pd
import os
import pickle
import xarray as xr

## Pysam

rundown on pysam...

https://nrel-pysam.readthedocs.io/en/main/inputs-from-sam.html

### PVGIS

Only works with PVGIS

In [ ]:
weather_pvgis, meta_pvgis = pvdeg.weather.get(
    database="PVGIS", id=(25.783388, -81.189029)
)

In [ ]:
results = pvdeg.pysam.pysam(
    weather_df=weather_pvgis,
    meta=meta_pvgis,
    pv_model="pysamv1",
    pv_model_default="FlatPlatePVCommercial",
)

In [ ]:
results["annual_energy"]

### Local Geospatial

I don't know why this works with NSRDB data but it does. May not work with PSM3 yet. Only NSRDB from HPC? How to fix.

In [ ]:
# this is just a wrapper to grab the result we want
def pysam_annual_energy(weather_df, meta, pv_model="pysamv1", pv_model_default="FlatPlatePVCommercial"):
    results = pvdeg.pysam.pysam(
        weather_df=weather_df,
        meta=meta,
        pv_model='pysamv1',
        pv_model_default=pv_model_default,
    )

    return results["annual_energy"]

In [ ]:
GEO_META = pd.read_csv(os.path.join(TEST_DATA_DIR, "summit-meta.csv"), index_col=0)

GEO_WEATHER = xr.open_dataset(os.path.join(TEST_DATA_DIR, "summit-weather.nc"))

template = pvdeg.geospatial.output_template(
    ds_gids=GEO_WEATHER,
    shapes={
        "Annual Energy": ("gid",),
    },
)

geo_res = pvdeg.geospatial.analysis(
    weather_ds=GEO_WEATHER,
    meta_df=GEO_META,
    func=pysam_annual_energy,
    template=template,
)

In [ ]:
geo_res

## NSRDB API

broken

weather_db = 'PSM3'
weather_id = (25.783388, -80.189029)
weather_arg = {'api_key': 'DEMO_KEY',
               'email': 'user@mail.com',
               'names': 'tmy',
               'attributes': [],
               'map_variables': True}

weather_df, meta = pvdeg.weather.get(weather_db, weather_id, **weather_arg)

### Geospatial Scenario

In [ ]:
location_grabber = pvdeg.scenario.GeospatialScenario()

location_grabber.addLocation(country="United States", downsample_factor=80)

In [ ]:
location_grabber.plot_coords()

In [ ]:
geo_weather, geo_meta = location_grabber.geospatial_data()

template = pvdeg.geospatial.output_template(
    ds_gids=geo_weather,
    shapes={
        "pysam_annual_energy": ("gid",),
    },
)

geo_res = pvdeg.geospatial.analysis(
    weather_ds=geo_weather,
    meta_df=geo_meta,
    func=pysam_annual_energy,  # using wrapper from before
    template=template,
)

In [ ]:
pvdeg.geospatial.plot_sparse_analysis(geo_res, data_var="pysam_annual_energy")

In [ ]:
# Check weather data time dimension compatibility
if 'time' in GEO_WEATHER.dims:
    times = pd.to_datetime(GEO_WEATHER['time'].values)
    years = times.year
    unique_years = set(years)
    if len(unique_years) != 1:
        print(f"Warning: Weather data contains multiple years: {unique_years}. Pysam expects a single year.")
    hours_per_year = times.size / len(unique_years)
    if hours_per_year not in [8760, 8784]:
        print(f"Warning: Unexpected number of timesteps per year: {hours_per_year}. Expected 8760 or 8784.")
else:
    print("Warning: No 'time' dimension found in weather data. Pysam may fail.")